# o²S²PARC module from the NIH SPARC Python Client


In this document you will learn how to access the [o²S²PARC Python Client](https://itisfoundation.github.io/osparc-simcore-clients/#/) as part fo the [NIH SPARC Python Client](https://github.com/nih-sparc/sparc.client).

### Pre-requites:
- Python version >=3.10: as requested by the [sparc.client](https://pypi.org/project/sparc.client/) package.
- An account on [osparc.io](https://osparc.io): you will need this to generate API tokens to use the o²S²PARC module. If you don't have an account, request one at oSPARC Support, as explained [here](https://sparc.science/resources/4LkLiH5s4FV0LVJd3htsvH).
- Have a look to the [NIH SPARC Python Client doc](https://github.com/nih-sparc/sparc.client/blob/main/docs/tutorial.ipynb) as a preliminary step (and learn how to set up the Configuration).

## Installation

In [1]:
# import the necessary packages
import importlib, getpass
# `is None: # This give and error (cannot find sparc)
# ! pip install sparc.client
try:
  import sparc.client
except ImportError:
  ! pip install git+https://github.com/nih-sparc/sparc.client
try:
  from tqdm import tqdm
except ImportError:
  ! pip install tqdm
from pprint import pprint
import os
from tqdm import tqdm
from sparc.client import SparcClient
from sparc.client.services.o2sparc import (
  O2SparcService,
  O2SparcSolver
)
from time import sleep
from pathlib import Path
from tempfile import TemporaryDirectory

/home/iavarone/.pyenv/versions/3.10.11/envs/sparc_client/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Login to the o²S²PARC Python Client

Set-up credentials for o²S²PARC in osparc.io as explained [here](https://docs.osparc.io/#/docs/platform_introduction/user_setup/security_details?id=generating-o%c2%b2s%c2%b2parc-tokens). Then use the "Key" as "Username" and "Secret" as "Password". Another option is to export them as environment variables.

In [11]:
os.environ["O2SPARC_HOST"] = getpass.getpass("osparc host, e.g. https://api.osparc.io")
os.environ["O2SPARC_USERNAME"] = getpass.getpass("API Key:")
os.environ["O2SPARC_PASSWORD"] = getpass.getpass("API Secret:")

osparc host, e.g. https://api.osparc.io ········
API Key: ········
API Secret: ········


In [17]:
assert "O2SPARC_HOST" in os.environ, "O2SPARC_HOST must be exposed as an environment variable"
assert "O2SPARC_USERNAME" in os.environ, "O2SPARC_USERNAME must be exposed as an environment variable"
assert "O2SPARC_PASSWORD" in os.environ, "O2SPARC_PASSWORD must be exposed as an environment variable"

Access to the computational services provided by the `o2sparc` submodule, is provided via `O2SparcService`, an instance of which can be creates as follows:

In [13]:
client = SparcClient(connect=False, config_file='config.ini')
o2sparc: O2SparcService = client.o2sparc

As a sanity check you can now contact the Osparc server to check that you are logged in as the correct user:

In [14]:
print(o2sparc.get_profile())
# foo@bar.com

iavarone@itis.swiss


## Run a computation

Now to the fun part :). In the following cell we specify a job (or a "computation") which we submit to a solver (or "computational service") on the Osparc platform. More information on computational services [here](https://docs.osparc.io/#/docs/platform_introduction/services?id=service-types).

In [15]:
with TemporaryDirectory() as tmp_dir:
  input_file: Path = Path(tmp_dir) / "input_file.txt"
  input_file.write_text("3")

  job: dict = {
    "input_3": 0,
    "input_2": 3.0,
    "input_1": input_file
  }

  solver: O2SparcSolver = o2sparc.get_solver(solver_key="simcore/services/comp/itis/sleeper",solver_version="2.0.2")

  job_id = solver.submit_job(job)

  pbar = tqdm(total=1.0)
  progress: float = 0
  while not solver.job_done(job_id):
    sleep(1)
    if solver.get_job_progress(job_id) > progress:
      pbar.update(solver.get_job_progress(job_id) - progress)
      progress = solver.get_job_progress(job_id)

  print("job outputs:")
  for output_name, result in solver.get_results(job_id).items():
    if isinstance(result,Path):
      print(f"{output_name}: {Path(result).read_text()}")
    else:
      print(f"{output_name}: {result}")


  print("job log:")
  log_dir: TemporaryDirectory = solver.get_job_log(job_id)
  for elm in Path(log_dir.name).rglob("*"):
    if elm.is_file():
      print(elm.read_text())
      

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.0/1.0 [00:05<00:00,  5.54s/it]

job outputs:
output_1: 8
output_2: 7.0
job log:
Entrypoint for stage production ...
User : uid=0(root) gid=0(root) groups=0(root)
Workdir : /home/scu
setting correct user id/group id...
Folder mounted owned by user 8004:8004-'scu'...
group already exists
adding scu to group scu...
Adding user `scu' to group `scu' ...
Adding user scu to group scu
Done.
changing scu:scu (8004:8004) to scu:scu (8004:8004)
usermod: no changes
Changing group properties of files around from 8004 to group scu
Changing ownership properties of files around from 8004 to group scu
Starting /bin/sh
-c
run ...
  scu rights    : uid=8004(scu) gid=8004(scu) groups=8004(scu)
  local dir : total 12
drwxr-xr-x 1 scu  scu   105 Aug 11  2020 .
drwxr-xr-x 1 root root   17 Aug 11  2020 ..
-rw-r--r-- 1 scu  scu   220 Aug 11  2020 .bash_logout
-rw-r--r-- 1 scu  scu  3771 Aug 11  2020 .bashrc
-rw-r--r-- 1 scu  scu   807 Aug 11  2020 .profile
drwxr-xr-x 1 scu  scu    27 Aug 11  2020 docker
drwxr-xr-x 1 scu  scu    35 Aug 11  20

## Understanding the steps

Let's break this into pieces. 
First we specify the job/input to our solver. A job for a `O2SparcSolver` is nothing but a dictionary whose entries are one of the following types: `str`, `int`, `float` or `pathlib.Path`. Only `pathlib.Path` objects can be passed which point to existing files, like our `input_1` above.

```python
  input_file: Path = Path(tmp_dir) / "input_file.txt"
  input_file.write_text("3")

  job: dict = {
    "input_3": 0,
    "input_2": 3.0,
    "input_1": input_file
  }
```

Second, we get the solver/computational resource we want to use by specifying its identifier and version. Once we have our solver we can submit the job to it.

```python
  solver: O2SparcSolver = o2sparc.get_solver(solver_key="simcore/services/comp/itis/sleeper",solver_version="2.0.2")

  job_id = solver.submit_job(job)
```

Next, we need to wait for the job to be solved

```python
 pbar = tqdm(total=1.0)
  progress: float = 0
  while not solver.job_done(job_id):
    sleep(1)
    if solver.get_job_progress(job_id) > progress:
      pbar.update(solver.get_job_progress(job_id) - progress)
      progress = solver.get_job_progress(job_id)
```

Once the solver is done performing its computation we can get the results it produces

```python
  print("job outputs:")
  for output_name, result in solver.get_results(job_id).items():
    if isinstance(result,Path):
      print(f"{output_name}: {Path(result).read_text()}")
    else:
      print(f"{output_name}: {result}")
```

as well as the log it has produced

```python
  print("job log:")
  log_dir: TemporaryDirectory = solver.get_job_log(job_id)
  for elm in Path(log_dir.name).rglob("*"):
    if elm.is_file():
      print(elm.read_text())
```

Getting the log can be particularly useful for determining issues encountered by the solver. E.g. if the specified `job` is invalid, so that the solver cannot use it, the log will typically indicate this.

## More resources
- More tutorials on the o²S²PARC Python Client are available [here](https://itisfoundation.github.io/osparc-simcore-clients/#/clients/python/docs/v0.5.0/README)
- Documentation on the NIH SPARC Client and its sub-modules can be found [here](https://docs.sparc.science/docs/sparc-python-client).